In [1]:
import os
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
from zipfile import ZipFile
from google.colab.patches import cv2_imshow
import tensorflow as tf
from tensorflow.keras import models,optimizers,layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from tensorflow.keras.callbacks import TensorBoard

from tensorflow.python.keras import layers 
from tensorflow.python.keras import models
from tensorflow.python.keras import optimizers
from keras.callbacks import TensorBoard
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import img_to_array
from tensorflow.keras.utils import load_img
from tensorflow.keras.utils import plot_model
import pydot_ng as pydot
from sklearn.model_selection  import train_test_split
import math

In [3]:
# Monter votre Drive afin de pouvoir y'acceder 
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Facial emotion recognition
The aim of this lab (TP) is to is to recognize facial emotions from a video using deep learning
approach. The system can take pictures or video form webcam as input. It detects all faces in
each frame, and then classifies the emotions as belonging to one of the 7 emotion categories:
Angry, Disgusted, Neutral, Sad, Happy, Surprised and Fear.
Facial landmarks are used to localize and represent salient regions of the face, such as: eyes,
eyebrows nose, mouth jawline. They have been successfully applied to face alignment, head
pose estimation, face swapping, blink detection and much more.

We propose here to develop an approach based on famous facial landmarks (68 facial landmarks can be detected with dlib library), which and use the Eucledian distance between them as 68x68 array and use them as feature to learn a network

In [4]:
def euclidean(a, b):
    dist = math.sqrt(math.pow((b[0] - a[0]), 2) + math.pow((b[1] - a[1]), 2))
    return dist 

def euclidean_all(a):  # calculates distances between all 68 elements
	distances = ""
	for i in range(0, len(a)):
		for j in range(0, len(a)):
			dist = euclidean(a[i], a[j])
			dist = "%.2f" % dist;
			distances = distances + " " + str(dist)
	return distances

In [5]:
	def euclidean_all_numpy(a):  # calculates distances between all 68 elements
		distances = []
		for i in range(0, len(a)):
			for j in range(0, len(a)):
				dist = euclidean(a[i], a[j])
				distances.append(dist)
		return np.array(distances)

To train this network, you can use the dataset (train_test_landmarks.zip extracted from a
mixture of CK+ and JAFFE datasets) already prepared as feature vector of distances with the
format:(741, 4624) for train and (365, 4624) for test. After the training, save you best model.
https://drive.google.com/file/d/1teun1oa2nolShwfpksV-nOdeWCX484ax/view?usp=sharing

In [6]:
model = models.Sequential()                                                              

model.add(layers.Dense(512, activation='relu',input_shape=(4624,)))
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(512, activation='relu'))

model.add(layers.Dense(7, activation='softmax')) 

model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['acc'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 512)               2368000   
_________________________________________________________________
dense_1 (Dense)              (None, 512)               262656    
_________________________________________________________________
dense_2 (Dense)              (None, 512)               262656    
_________________________________________________________________
dense_3 (Dense)              (None, 512)               262656    
_________________________________________________________________
dense_4 (Dense)              (None, 512)               262656    
_________________________________________________________________
dense_5 (Dense)              (None, 7)                 3591      
Total params: 3,422,215
Trainable params: 3,422,215
Non-trainable params: 0
______________________________________________

In [7]:
from zipfile import ZipFile
file_name = "/content/drive/MyDrive/DeepLearning_SDI/facial_ressources/Ressources.zip"

with ZipFile(file_name, 'r') as zip:
  zip.extractall()
  print('Done')

Done


In [8]:
from zipfile import ZipFile
file_name = "./Ressources/train_test_landmarks.zip"

with ZipFile(file_name, 'r') as zip:
  zip.extractall()
  print('Done')

file_name = "/content/Ressources/train_test_landmarks.zip"

with ZipFile(file_name, 'r') as zip:
  zip.extractall()
  print('Done')

Done
Done


In [9]:
X_test= np.load('./x_test.npy')
X_train= np.load('./x_train.npy')
y_test = np.load('./y_test.npy')
y_train = np.load('./y_train.npy')

In [10]:
print(X_test.shape)
print(X_train.shape)
print(y_test.shape)
print(y_train.shape)

(365, 4624)
(741, 4624)
(365, 7)
(741, 7)


In [11]:
batch_size = 32
NAME = "TENSORBOARD_EMOTION"
tensorboard = TensorBoard(log_dir="./logs/{}".format(NAME))

In [12]:
%load_ext tensorboard
import datetime
logs_base_dir = "./logs"
os.makedirs(logs_base_dir, exist_ok=True)
%tensorboard --logdir {logs_base_dir}

Launching TensorBoard...

KeyboardInterrupt: ignored

history_Dense= model.fit(X_train,y_train,epochs=32,validation_data=(X_test,y_test),batch_size=batch_size,callbacks=[tensorboard])

In [13]:
checkpoint_filepath = './tmp/checkpoint'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)

In [14]:
history = model.fit(X_train, y_train,
batch_size=batch_size,
epochs=100,
validation_data=(X_test, y_test),
verbose=1,
callbacks=[tensorboard, model_checkpoint_callback])

Epoch 1/100
24/24 [==============================] - 2s 48ms/step - loss: 1.7056 - acc: 0.5074 - val_loss: 1.3295 - val_acc: 0.6027


Epoch 2/100
24/24 [==============================] - 1s 34ms/step - loss: 1.1768 - acc: 0.6302 - val_loss: 1.2051 - val_acc: 0.6658


Epoch 3/100
24/24 [==============================] - 1s 37ms/step - loss: 0.9823 - acc: 0.6896 - val_loss: 0.7780 - val_acc: 0.7589


Epoch 4/100
24/24 [==============================] - 1s 37ms/step - loss: 0.8619 - acc: 0.7152 - val_loss: 0.8157 - val_acc: 0.7096


Epoch 5/100
24/24 [==============================] - 1s 36ms/step - loss: 0.8118 - acc: 0.7314 - val_loss: 0.7433 - val_acc: 0.7562


...

Epoch 96/100
24/24 [==============================] - 1s 37ms/step - loss: 0.1216 - acc: 0.9528 - val_loss: 0.7260 - val_acc: 0.8521


Epoch 97/100
24/24 [==============================] - 1s 37ms/step - loss: 0.1996 - acc: 0.9285 - val_loss: 0.5690 - val_acc: 0.8658


Epoch 98/100
24/24 [==============================] - 1s 37ms/step - loss: 0.1394 - acc: 0.9420 - val_loss: 0.8840 - val_acc: 0.8055


Epoch 99/100
24/24 [==============================] - 1s 36ms/step - loss: 0.0988 - acc: 0.9636 - val_loss: 0.7227 - val_acc: 0.8411


Epoch 100/100
24/24 [==============================] - 1s 38ms/step - loss: 0.2176 - acc: 0.9217 - val_loss: 0.6944 - val_acc: 0.7945


In [15]:
model.save_weights('model_DENSE_weights.h5')
model.save('model_DENSE.h5')

In [ ]:
#On charge le modèle avec les meilleurs weights
model.load_weights(checkpoint_filepath)

Now, in order to use the vector of these distances to classify emotions from 
an image, you detect the face region, extract the facial landmarks and then compute the 4624 distances between them.

The following script allow to capture the face region using Haar detector
(haarcascade_frontalface_default.xml) and detect 68 facial landmarks using the learned model (shape_predictor_68_face_landmarks.dat):

In [17]:
import cv2
from imutils import face_utils
import imutils
import dlib
# -----------

face_cascade = cv2.CascadeClassifier('./Ressources/haarcascade_frontalface_default.xml')
# initialize dlib's face detector and create a predictor
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("./Ressources/shape_predictor_68_face_landmarks.dat")

# Detection via des images exemples

In [18]:
import matplotlib.pyplot as plt
from google.colab.patches import cv2_imshow

def detect_parts(image):
    # resize the image, and convert it to grayscale
    image = imutils.resize(image, width=200, height=200)

    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # detect faces in the grayscale image
    rects = detector(gray, 1)

    # loop over the face detections
    shape = None
    for (i, rect) in enumerate(rects):
        shape = predictor(gray, rect)
        shape = face_utils.shape_to_np(shape)
        
        # visualize all facial landmarks with a transparent overlay
        #output = face_utils.visualize_facial_landmarks(image, shape)
        #cv2.imshow("Image", output)
        #cv2.waitKey(0)
    return shape

img = cv2.imread("./example_image.png")
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
faces = face_cascade.detectMultiScale(gray, 1.3, 5)

emotions = ('Angry', 'Disgust', 'Fear', 'Happy', 'Neutral', 'Sad', 'Surprise')
for (x, y, w, h) in faces:
    cv2.rectangle(img, (x, y), (x + w, y + h), (255, 0, 0), 2)  # draw rectangle to main image
    detected_face = img[int(y):int(y + h), int(x):int(x + w)]  # crop detected face
    distances = euclidean_all_numpy(detect_parts(detected_face))
    predictions = model.predict(distances.reshape(1,-1))
    emotion = emotions[np.argmax(predictions)]

    # write emotion text above rectangle
    cv2.putText(img, emotion, (int(x), int(y)), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

cv2_imshow(img)
cv2.waitKey(0)

error: ignored

In [ ]:
img = cv2.imread("./example_image2.jpg")
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
faces = face_cascade.detectMultiScale(gray, 1.3, 5)

emotions = ('Angry', 'Disgust', 'Fear', 'Happy', 'Neutral', 'Sad', 'Surprise')
for (x, y, w, h) in faces:
    cv2.rectangle(img, (x, y), (x + w, y + h), (255, 0, 0), 2)  # draw rectangle to main image
    detected_face = img[int(y):int(y + h), int(x):int(x + w)]  # crop detected face
    distances = euclidean_all_numpy(detect_parts(detected_face))
    predictions = model.predict(distances.reshape(1,-1))
    emotion = emotions[np.argmax(predictions)]

    # write emotion text above rectangle
    cv2.putText(img, emotion, (int(x), int(y)), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

cv2_imshow(img)
cv2.waitKey(0)

In [ ]:
img = cv2.imread("./example_image3.webp")
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
faces = face_cascade.detectMultiScale(gray, 1.3, 5)

emotions = ('Angry', 'Disgust', 'Fear', 'Happy', 'Neutral', 'Sad', 'Surprise')
for (x, y, w, h) in faces:
    cv2.rectangle(img, (x, y), (x + w, y + h), (255, 0, 0), 2)  # draw rectangle to main image
    detected_face = img[int(y):int(y + h), int(x):int(x + w)]  # crop detected face
    distances = euclidean_all_numpy(detect_parts(detected_face))
    predictions = model.predict(distances.reshape(1,-1))
    emotion = emotions[np.argmax(predictions)]

    # write emotion text above rectangle
    cv2.putText(img, emotion, (int(x), int(y)), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

cv2_imshow(img)
cv2.waitKey(0)

error: ignored

To classify emotions in real time capture, you need first to capture your video streaming from a webcam:

In [20]:
cap = cv2.VideoCapture(0)
while (True):
 ret, img = cap.read()
 gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
 faces = face_cascade.detectMultiScale(gray, 1.3, 5)
 ...
 ...
 ...

# kill open cv things
cap.release()
cv2.destroyAllWindows()

error: ignored

# Questions : 

**Question 1**: Develop your emotion recognition system to perform the recognition in real time.


**Question 2**: Propose another model based on a CNN extracted directly from the frame images.
To do this, uses the subsets (train_test_faceImages.zip extracted from FER2013 dataset.

# Question 1

In [21]:
# import dependencies
from IPython.display import display, Javascript, Image
from google.colab.output import eval_js
from base64 import b64decode, b64encode
import cv2
import numpy as np
import PIL
import io
import html
import time
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode

In [22]:

def take_photo(filename='./photo.jpg', quality=0.8):
  js = Javascript('''
    async function takePhoto(quality) {
      const div = document.createElement('div');
      const capture = document.createElement('button');
      capture.textContent = 'Capture';
      div.appendChild(capture);

      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      // Wait for Capture to be clicked.
      await new Promise((resolve) => capture.onclick = resolve);

      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getVideoTracks()[0].stop();
      div.remove();
      return canvas.toDataURL('image/jpeg', quality);
    }
    ''')
  display(js)
  data = eval_js('takePhoto({})'.format(quality))
  binary = b64decode(data.split(',')[1])
  with open(filename, 'wb') as f:
    f.write(binary)
  return filename

In [ ]:
while (True):
  filename = take_photo()
  img = cv2.imread(filename)
  gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
  faces = face_cascade.detectMultiScale(gray, 1.3, 5)
  emotions = ('Angry', 'Disgust', 'Fear', 'Happy', 'Neutral', 'Sad', 'Surprise')
  for (x, y, w, h) in faces:
      cv2.rectangle(img, (x, y), (x + w, y + h), (255, 0, 0), 2)  # draw rectangle to main image
      detected_face = img[int(y):int(y + h), int(x):int(x + w)]  # crop detected face
      distances = euclidean_all_numpy(detect_parts(detected_face))
      predictions = model.predict(distances.reshape(1,-1))
      emotion = emotions[np.argmax(predictions)]

      # write emotion text above rectangle
      cv2.putText(img, emotion, (int(x), int(y)), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
      cv2_imshow(img)
# kill open cv things
cap.release()
cv2.destroyAllWindows()

In [24]:
# function to convert the JavaScript object into an OpenCV image
def js_to_image(js_reply):
  """
  Params:
          js_reply: JavaScript object containing image from webcam
  Returns:
          img: OpenCV BGR image
  """
  # decode base64 image
  image_bytes = b64decode(js_reply.split(',')[1])
  # convert bytes to numpy array
  jpg_as_np = np.frombuffer(image_bytes, dtype=np.uint8)
  # decode numpy array into OpenCV BGR image
  img = cv2.imdecode(jpg_as_np, flags=1)

  return img

# function to convert OpenCV Rectangle bounding box image into base64 byte string to be overlayed on video stream
def bbox_to_bytes(bbox_array):
  """
  Params:
          bbox_array: Numpy array (pixels) containing rectangle to overlay on video stream.
  Returns:
        bytes: Base64 image byte string
  """
  # convert array into PIL image
  bbox_PIL = PIL.Image.fromarray(bbox_array, 'RGBA')
  iobuf = io.BytesIO()
  # format bbox into png for return
  bbox_PIL.save(iobuf, format='png')
  # format return string
  bbox_bytes = 'data:image/png;base64,{}'.format((str(b64encode(iobuf.getvalue()), 'utf-8')))

  return bbox_bytes

In [25]:
# JavaScript to properly create our live video stream using our webcam as input
def video_stream():
  js = Javascript('''
    var video;
    var div = null;
    var stream;
    var captureCanvas;
    var imgElement;
    var labelElement;
    
    var pendingResolve = null;
    var shutdown = false;
    
    function removeDom() {
       stream.getVideoTracks()[0].stop();
       video.remove();
       div.remove();
       video = null;
       div = null;
       stream = null;
       imgElement = null;
       captureCanvas = null;
       labelElement = null;
    }
    
    function onAnimationFrame() {
      if (!shutdown) {
        window.requestAnimationFrame(onAnimationFrame);
      }
      if (pendingResolve) {
        var result = "";
        if (!shutdown) {
          captureCanvas.getContext('2d').drawImage(video, 0, 0, 640, 480);
          result = captureCanvas.toDataURL('image/jpeg', 0.8)
        }
        var lp = pendingResolve;
        pendingResolve = null;
        lp(result);
      }
    }
    
    async function createDom() {
      if (div !== null) {
        return stream;
      }

      div = document.createElement('div');
      div.style.border = '2px solid black';
      div.style.padding = '3px';
      div.style.width = '100%';
      div.style.maxWidth = '600px';
      document.body.appendChild(div);
      
      const modelOut = document.createElement('div');
      modelOut.innerHTML = "<span>Status:</span>";
      labelElement = document.createElement('span');
      labelElement.innerText = 'No data';
      labelElement.style.fontWeight = 'bold';
      modelOut.appendChild(labelElement);
      div.appendChild(modelOut);
           
      video = document.createElement('video');
      video.style.display = 'block';
      video.width = div.clientWidth - 6;
      video.setAttribute('playsinline', '');
      video.onclick = () => { shutdown = true; };
      stream = await navigator.mediaDevices.getUserMedia(
          {video: { facingMode: "environment"}});
      div.appendChild(video);

      imgElement = document.createElement('img');
      imgElement.style.position = 'absolute';
      imgElement.style.zIndex = 1;
      imgElement.onclick = () => { shutdown = true; };
      div.appendChild(imgElement);
      
      const instruction = document.createElement('div');
      instruction.innerHTML = 
          '<span style="color: red; font-weight: bold;">' +
          'When finished, click here or on the video to stop this demo</span>';
      div.appendChild(instruction);
      instruction.onclick = () => { shutdown = true; };
      
      video.srcObject = stream;
      await video.play();

      captureCanvas = document.createElement('canvas');
      captureCanvas.width = 640; //video.videoWidth;
      captureCanvas.height = 480; //video.videoHeight;
      window.requestAnimationFrame(onAnimationFrame);
      
      return stream;
    }
    async function stream_frame(label, imgData) {
      if (shutdown) {
        removeDom();
        shutdown = false;
        return '';
      }

      var preCreate = Date.now();
      stream = await createDom();
      
      var preShow = Date.now();
      if (label != "") {
        labelElement.innerHTML = label;
      }
            
      if (imgData != "") {
        var videoRect = video.getClientRects()[0];
        imgElement.style.top = videoRect.top + "px";
        imgElement.style.left = videoRect.left + "px";
        imgElement.style.width = videoRect.width + "px";
        imgElement.style.height = videoRect.height + "px";
        imgElement.src = imgData;
      }
      
      var preCapture = Date.now();
      var result = await new Promise(function(resolve, reject) {
        pendingResolve = resolve;
      });
      shutdown = false;
      
      return {'create': preShow - preCreate, 
              'show': preCapture - preShow, 
              'capture': Date.now() - preCapture,
              'img': result};
    }
    ''')

  display(js)
  
def video_frame(label, bbox):
  data = eval_js('stream_frame("{}", "{}")'.format(label, bbox))
  return data

In [ ]:
# start streaming video from webcam
video_stream()
# label for video
label_html = 'Capturing...'
# initialze bounding box to empty
bbox = ''
count = 0 
emotions = ('Angry', 'Disgust', 'Fear', 'Happy', 'Neutral', 'Sad', 'Surprise')
while True:
    js_reply = video_frame(label_html, bbox)
    if not js_reply:
        break

    # convert JS response to OpenCV Image
    img = js_to_image(js_reply["img"])

    # create transparent overlay for bounding box
    bbox_array = np.zeros([480,640,4], dtype=np.uint8)

    # grayscale image for face detection
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

    # get face region coordinates
    faces = face_cascade.detectMultiScale(gray)
    # get face bounding box for overlay
    for (x,y,w,h) in faces:
      bbox_array = cv2.rectangle(bbox_array,(x,y),(x+w,y+h),(255,0,0),2)
      detected_face = img[int(y):int(y + h), int(x):int(x + w)]  # crop detected face
      shape = detect_parts(detected_face)
      if shape is not None:
        distances = euclidean_all_numpy(shape)
        predictions = model.predict(distances.reshape(1,-1), verbose=0)
        emotion = emotions[np.argmax(predictions)]

        bbox_array = cv2.putText(bbox_array, emotion, (int(x), int(y)), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

    bbox_array[:,:,3] = (bbox_array.max(axis = 2) > 0 ).astype(int) * 255
    # convert overlay of bbox into bytes
    bbox_bytes = bbox_to_bytes(bbox_array)
    # update bbox so next frame gets new overlay
    bbox = bbox_bytes

# Question 2

In [ ]:
X_train = np.load('./images_train.npy')
rgb_X_train = np.repeat(X_train.reshape(X_train.shape[:-1])[..., np.newaxis], 3, -1)
X_test = np.load('./images_test.npy')
rgb_X_test = np.repeat(X_test.reshape(X_test.shape[:-1])[..., np.newaxis], 3, -1)
y_train = np.load('./labels_train.npy')
y_test = np.load('./labels_test.npy')

In [ ]:
train_datagen = ImageDataGenerator(
rotation_range=40,
width_shift_range=0.2,
height_shift_range=0.2,
shear_range=0.2,
zoom_range=0.2,
horizontal_flip=True,)
val_datagen = ImageDataGenerator() #We do not augment validation data. we only perform rescale

In [ ]:
train_generator = train_datagen.flow(X_train, y_train,batch_size=batch_size)
val_generator = val_datagen.flow(X_test, y_test, batch_size=batch_size)

In [ ]:
CNN = models.Sequential() # (1)
CNN.add(layers.Conv2D(32, (3, 3), activation='relu',input_shape=X_train.shape[1:]))#(2)
CNN.add(layers.MaxPooling2D((2, 2)))
CNN.add(layers.Conv2D(64, (3, 3), activation='relu'))
CNN.add(layers.MaxPooling2D((2, 2))) # (3)
CNN.add(layers.Conv2D(128, (3, 3), activation='relu'))
CNN.add(layers.MaxPooling2D((2, 2)))
CNN.add(layers.Conv2D(128, (3, 3), activation='relu'))
CNN.add(layers.MaxPooling2D((2, 2)))
CNN.add(layers.Flatten()) # (4)
CNN.add(layers.Dropout(0.5)) #Dropout for regularization # (5)
CNN.add(layers.Dense(512, activation='relu'))
CNN.add(layers.Dense(y_train.shape[1], activation='softmax'))

CNN.summary()

In [ ]:
CNN.compile(loss='categorical_crossentropy',optimizer="rmsprop", metrics=['accuracy'])

In [ ]:
NAME = "CNN_TENSORBOARD"
tensorboard = TensorBoard(log_dir="./logs/{}".format(NAME))

In [ ]:
checkpoint_filepath = './tmp/CNN/checkpoint'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)

In [ ]:
history = CNN.fit(train_generator,
epochs=20,
validation_data=val_generator,
callbacks=[tensorboard, model_checkpoint_callback])

In [ ]:
train_datagen = ImageDataGenerator(rescale=255,
rotation_range=40,
width_shift_range=0.2,
height_shift_range=0.2,
shear_range=0.2,
zoom_range=0.2,
horizontal_flip=True,)
val_datagen = ImageDataGenerator(rescale=255) #We do not augment validation data. we only perform rescale

In [ ]:
batch_size = 32

train_generator = train_datagen.flow(rgb_X_train, y_train,batch_size=batch_size)
val_generator = val_datagen.flow(rgb_X_test, y_test, batch_size=batch_size)

In [ ]:
i = tf.keras.layers.Input([None, None, 3], dtype = tf.float32)
x = tf.keras.layers.Lambda(lambda image: tf.image.resize(image, (224,224)))(i)
x = tf.keras.applications.vgg16.preprocess_input(x)
core = tf.keras.applications.vgg16.VGG16()
for l in core.layers:
  l.trainable = False
x = core(x)
# Stacking a new simple convolutional network on top of it
#x = layers.Conv2D(64, (3, 3), activation='relu')(x)
#x = layers.MaxPooling2D(pool_size=(2, 2))(x)
#x = layers.Flatten()(x)
x = layers.Dense(256, activation='relu')(x)
x = layers.Dense(y_train.shape[1], activation='softmax')(x)

In [ ]:
CNN_from_VGG = tf.keras.Model(inputs=i, outputs=x)

In [ ]:
CNN_from_VGG.summary()

In [ ]:
CNN_from_VGG.compile(loss='categorical_crossentropy',optimizer="adam", metrics=['accuracy'])

In [ ]:
NAME = "CNN_from_VGG"
tensorboard = TensorBoard(log_dir="./logs/{}".format(NAME))

In [ ]:
checkpoint_filepath = './tmp/CNN_from_VGG/checkpoint'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)

In [ ]:
history = CNN_from_VGG.fit(train_generator,
epochs=20,
validation_data=val_generator,
callbacks=[tensorboard, model_checkpoint_callback])